In [2]:
import pandas as pd
import numpy as np
import wget

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 50)

import xgboost as xgb
import sklearn

print("XGB Version          : ", xgb.__version__)
print("Scikit-Learn Version : ", sklearn.__version__)

XGB Version          :  2.1.2
Scikit-Learn Version :  1.4.2


In [3]:
data0 = pd.read_csv("verified_online.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8855181,https://sparkflaretrust.com/connect.html,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:46:02+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
1,8855180,https://llltreuirzleeno.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:15+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
2,8855179,http://llltreuirzleeno.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:14+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
3,8855176,http://hubtrezorstart.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
4,8855177,https://hubtrezorstart.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other


In [4]:
data0.shape


(67678, 8)

In [5]:
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8717279,https://telegra.ph/BT-Business-Communications-...,http://www.phishtank.com/phish_detail.php?phis...,2024-08-16T15:47:59+00:00,yes,2024-08-16T15:53:35+00:00,yes,Other
1,8376431,https://new.express.adobe.com/webpage/pZDKTv9g...,http://www.phishtank.com/phish_detail.php?phis...,2023-11-27T15:31:47+00:00,yes,2023-11-27T15:43:19+00:00,yes,Other
2,8831188,http://suall4ss.top,http://www.phishtank.com/phish_detail.php?phis...,2024-11-01T19:50:22+00:00,yes,2024-11-01T20:02:24+00:00,yes,Other
3,8836530,https://jawatan-kosong.foie.site/04fe663632581...,http://www.phishtank.com/phish_detail.php?phis...,2024-11-06T12:02:16+00:00,yes,2024-11-08T08:51:47+00:00,yes,Other
4,8468077,https://docs.google.com/presentation/d/e/2PACX...,http://www.phishtank.com/phish_detail.php?phis...,2024-02-26T23:11:40+00:00,yes,2024-02-27T10:42:55+00:00,yes,Other


In [6]:
phishurl.shape

(5000, 8)

In [7]:
#legitimate URL
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [8]:
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [9]:
legiurl.shape


(5000, 1)

In [10]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [11]:
# 1.Domain of the URL (Domain) 
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [12]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [13]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

In [14]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [15]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [16]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [17]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [18]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [19]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [20]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
     

In [21]:
!pip install python-whois


In [22]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
import csv
from datetime import datetime


In [23]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

In [24]:
def web_traffic(url, csv_file):
  # Read CSV data into a dictionary for efficient lookup
  rank_data = {}
  try:
      with open(csv_file, 'r') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
              if len(row) >= 2:
                  rank_data[row[1]] = int(row[0])
              else:
                  print(f"Warning: Invalid row in CSV: {row}")
  except FileNotFoundError:
      print(f"CSV file not found: {csv_file}")
      return 0
  except csv.Error as e:
      print(f"Error reading CSV file: {e}")
      return 0

  # Extract domain from URL
  domain = url.split("//")[1].split("/")[0]

  # Check if domain exists in the CSV
  if domain in rank_data:
      rank = rank_data[domain]
      if rank < 100000:
          return 1
      else:
          return 0
  else:
      # Domain not found in CSV, return 1 if rank is assumed to be low
      return 1

In [25]:
# 12.Web traffic (Web_Traffic)
def check_rank(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [26]:
# 13.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [27]:
# 14.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

In [28]:
#HTML and JavaScript based Features
import requests

In [29]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("", response.text):
      return 1
    else:
      return 0

In [30]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[|]", response.text):
          return 0
      else:
          return 1

In [31]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [32]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [33]:
#Function to extract features
def featureExtraction(url,label):
  #print('url', url,'label', label)
  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  # features.append(web_traffic(url))
  features.append(web_traffic(url,'top-1m.csv'))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  #HTML & Javascript based features (4)
  try:
    response = requests.get(url)  
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features

In [34]:
#Legitimate URLs
legiurl.shape

(5000, 1)

In [ ]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(0, 5000):
    url = legiurl['URLs'][i]
    legi_features.append(featureExtraction(url,label))

In [ ]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()